# RAG - Metrics & Evaluation

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20RAG_Metrics%26Evaluation.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

## RAG (and LLM) Metrics

Evaluating LLMs and retrieval-augmented generation (RAG) systems requires a detailed analysis of individual components and the system as a whole. Setting baseline values for pieces, such as chunking logic and embedding models, then assessing each part individually and end-to-end is critical for understanding the impact of changes on the system’s overall performance. Holistic modules in these evaluations don’t always require ground-truth labels, as they can be assessed based on the language model’s query, context, response, and interpretations.

Here are five commonly used metrics for evaluating your systems:

1.  [Correctness](https://docs.llamaindex.ai/en/latest/examples/evaluation/correctness_eval.html): This metric assesses whether the generated answer aligns with a given query’s reference answer. It requires labels and involves verifying the accuracy of the generated answer by comparing it to a predefined reference answer.
2.  [Faithfulness](https://docs.llamaindex.ai/en/latest/examples/evaluation/faithfulness_eval.html): This evaluates the integrity of the answer concerning the retrieved contexts. The faithfulness metric ensures that the answer accurately reflects the information in the retrieved context, free from distortions or fabrications that might misrepresent the source material.
3.  [Context Relevancy](https://docs.llamaindex.ai/en/latest/examples/evaluation/relevancy_eval.html): This measures the relevance of the retrieved context and the resulting answer to the original query. The goal is to ensure the system retrieves relevant information to the user’s request.
4.  [Guideline Adherence](https://docs.llamaindex.ai/en/latest/examples/evaluation/guideline_eval.html): This metric determines if the predicted answer follows established guidelines. It checks whether the response meets predefined criteria, including stylistic, factual, and ethical standards, ensuring the answer responds to the query and adheres to specific norms.
5.  [Embedding Semantic Similarity](https://docs.llamaindex.ai/en/latest/examples/evaluation/semantic_similarity_eval.html): This involves calculating a similarity score between the generated and reference answers’ embeddings. It requires reference labels and helps estimate the closeness of the generated response to a reference in terms of semantic content.

💡You can find the documentation pages for the above metrics at [towardsai.net/book.](http://towardsai.net/book)

The evaluation of RAG applications begins with a focus on their primary objective: generate useful outputs supported by contextually relevant facts obtained from retrievers. This analysis then zooms in on specific evaluation metrics such as faithfulness, answer relevancy, and the  [Sensibleness and Specificity Average (SSA)](https://arxiv.org/abs/2001.09977). Google’s SSA metric, which assesses open-domain chatbot responses, evaluates sensibleness (contextual coherence) and specificity (providing detailed and direct responses). Initially involving human evaluators, this metric ensures that outputs are comprehensive and not excessively vague or general.

A high faithfulness score does not necessarily correlate with high relevance. For instance, a response that accurately mirrors the context but does not directly address the query would receive a lower score in answer relevance. This could happen mainly if the response contains incomplete or redundant elements, indicating a gap between the accuracy of the provided context and the direct relevance to the question.

This section proceeds by showing how to compute the faithfulness score using the `FaithfulnessEvaluator` class from LlamaIndex. This metric focuses on preventing the issue of “hallucination” and examines responses to determine their alignment with the retrieved context. It assesses whether the response is consistent with the context and the initial query and fits with reference answers or guidelines. The output of this evaluation is a boolean value, indicating whether the response has successfully met the criteria for accuracy and faithfulness.

To use the FaithfulnessEvaluator, install the required libraries using Python’s package manager using !pip install -q llama-index==0.9.14.post3 deeplake==3.8.12 openai==1.3.8 cohere==4.37.

Following this, set the API keys for OpenAI and Activeloop and replace the placeholders in the provided code with their respective API keys.

In [ ]:
import os
from advanced_rag_custom_utils.helper import get_openai_api_key, get_activeloop_api_key, get_cohere_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()
COHERE_API_KEY = get_cohere_api_key()

Here’s an example for evaluating a single response for faithfulness:

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

# build service context
llm = OpenAI(model="gpt-4-turbo", temperature=0.0)
service_context = ServiceContext.from_defaults(llm=llm)

from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

vector_store = DeepLakeVectorStore(
dataset_path="hub://genai360/LlamaIndex_paulgraham_essay", overwrite=False
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

from llama_index.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(service_context=service_context)

# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What does Paul Graham do?"
)

eval_result = evaluator.evaluate_response(response=response)

print("> response:", response)
print("> evaluator result:", eval_result.passing)

> response: Paul Graham is involved in various activities. He is a writer and has given talks on topics such as starting a startup. He has also worked on software development, including creating software for generating websites and building online stores. Additionally, he has been a studio assistant for a beloved teacher who is a painter.  
> evaluator result: True

Most of the above code was previously discussed and will likely be familiar. It includes creating an index from the Deep Lake vector database, using it to make queries to the LLM, and performing the evaluation procedure. In this process, the query engine answers the question and sends its response to the evaluator for further examination.

In this case, the code creates a `FaithfulnessEvaluator` object, a mechanism for evaluating the precision of responses produced by the language model, GPT-4 Turbo. This evaluator operates using the previously defined `service_context`, which contains the configured GPT-4 Turbo model and provides the settings and parameters required for the language model’s optimal functioning.

The fundamental responsibility of the `FaithfulnessEvaluator` is assessing the extent to which the responses from the language model align with accurate and reliable information. It uses a series of criteria or algorithms to accomplish this, comparing the model-generated responses with verified factual data or anticipated results.

The evaluator proceeds to examine the response for its commitment to factual accuracy. This involves determining if the response accurately and dependably represents historical facts related to the query. The outcome of this assessment (`eval_result`) is then reviewed to ascertain if it aligns with the accuracy benchmarks established by the evaluator, as denoted by `eval_result.passing`. The result returns a Boolean (here `True`) value indicating whether the response passed the accuracy and faithfulness checks.

## Retrieval-Specific Evaluation Metrics

The evaluation of retrieval in RAG systems (not just LLM generation quality) involves determining the relevance of documents to specific queries. In information retrieval, the main goal is to identify unstructured data that meets a particular information requirement within a database.

>✴️ RAG system outputs need three critical aspects: factual accuracy, direct relevance to the query, and inclusion of essential contextual information.

![image](evaluation-metrics-of-retrieval-in-rag.jpg)
##### *The evaluation metrics of retrieval in RAG systems*.

Metrics used to assess the effectiveness of a retrieval system mostly include Mean Reciprocal Rank (MRR), Hit Rate, Mean Average Precision (MAP), and Normalized Discounted Cumulative Gain (NDCG):

-   **Mean Reciprocal Rank (MRR)**  measures the effectiveness of a system by averaging the reciprocal of the rank of the first relevant item in the search results. Higher MRR indicates better performance, as relevant items appear earlier.
-   **Hit Rate**  evaluates how often a relevant item appears within the top N results. It is a simple accuracy measure indicating the presence of at least one relevant item in the retrieved set.
-   **Mean Average Precision (MAP)**  calculates the average precision at each relevant item across multiple queries. It provides a single measure of quality by considering the precision of relevant items’ placements within the ranked results.
-   **Normalized Discounted Cumulative Gain (NDCG)**  evaluates the quality of the ranking by considering both the position and relevance of items. It discounts the relevance of items lower in the ranking, rewarding systems that rank highly relevant items earlier.

The `RetrieverEvaluator` from LlamaIndex is an advanced method to calculate metrics such as Mean Reciprocal Rank (MRR) and Hit Rate. Its primary function is to evaluate the effectiveness of a retrieval system that sources data relevant to user queries from a database or index. This class measures the retriever’s performance in responding to specific questions and providing the expected results, setting benchmarks for assessment.

For this evaluator, it is necessary to compile an evaluation dataset, which includes the content, a set of queries, and corresponding reference points for answering these queries. The `generate_question_context_pairs` function in LlamaIndex can create the evaluation dataset automatically. The process involves inputting a query and using the dataset as a benchmark to ensure the retrieval system accurately sources the correct documents. A detailed [tutorial on using the](https://docs.llamaindex.ai/en/stable/examples/evaluation/retrieval/retriever_eval.html) RetrieverEvaluator from the LlamaIndex documentation is accessible at [towardsai.net/book](http://towardsai.net/book).

>💡Although the evaluation of single queries is discussed, real-world applications typically need batch evaluations. This involves extracting a broad range of queries and their anticipated outcomes from the retriever to assess its general reliability. The retriever is evaluated using multiple queries and the expected results during batch testing. The process involves methodically inputting various queries into the retriever and comparing its responses to established correct answers to measure its consistency accurately.

There are open-source datasets that can be freely used to evaluate a RAG pipeline. An evaluation dataset comprises a selection of queries carefully paired with the most appropriate sources containing their answers. Such a dataset includes ideal expected answers from the large language model. Each query is paired with the most relevant source from the document, ensuring these sources directly address the queries.

Developing an evaluation dataset requires collecting a variety of realistic customer queries and matching them with expert answers. This dataset can be used to evaluate the responses of a language model, ensuring the LLM’s answers are accurate and relevant compared to expert responses. More information on  [creating this dataset](https://github.com/microsoft/promptflow-resource-hub/blob/main/sample_gallery/golden_dataset/copilot-golden-dataset-creation-guidance.md)  is accessible at  [towardsai.net/book](http://towardsai.net/book).

Once the evaluation dataset is prepared, it can be utilized to assess the quality of responses from the LLM. Following each evaluation, metrics will be available to quantify the user experience, providing valuable insights into the performance of the LLM. For example:

![image](assess-quality.jpg)

>⚠️ Although generating questions synthetically has its advantages, it is advisable to use questions that are tied to authentic user experiences. The questions employed in these evaluations should closely resemble real user queries, which are usually difficult to accomplish with the large language model. It is recommended to manually create questions emphasizing the users’ viewpoint for a more precise representation.

## Community-Based Evaluation Tools

LlamaIndex includes various evaluation tools designed to encourage community engagement and collaborative efforts. These tools facilitate a collective process of assessing and improving the system. LlamaIndex enables the easy integration of constant feedback, allowing users and developers to participate actively in the evaluation phase. Notable tools in this ecosystem include  [Ragas](https://github.com/explodinggradients/ragas/blob/main/docs/howtos/integrations/llamaindex.ipynb), an important tool with extensive metrics for assessing and integrating with LlamaIndex, and DeepEval, a tool for in-depth review and complete assessments of several areas of the system.

### Evaluating with Ragas

Ragas’s evaluation process revolves around utilizing specific metrics, including faithfulness, answer relevancy, context precision, context recall, and harmfulness. This process has several key elements. The performance of the query engine is the central focus of the assessment, as it plays a crucial role in the evaluation. Ragas provides a range of metrics specifically designed to facilitate a comprehensive analysis of the engine’s capabilities. Finally, a carefully curated set of questions is used to test the engine’s proficiency in retrieving and generating accurate responses.

The first step to using the Ragas library is setting up a query engine. This process involves loading a document. We will use the “New York City” Wikipedia page as the source document in this case. Next, you will need to install two more libraries: one to process the webpage content, and the other is a prerequisite for the evaluation library: `!pip install html2text==2020.1.16 ragas==0.0.22`.

Load the content by passing a URL to the `SimpleWebPageReader` class. Use these documents to build the index and query engine. Now, you can try asking questions about the document!

In [ ]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index import VectorStoreIndex, ServiceContext

documents = SimpleWebPageReader(html_to_text=True).load_data(
["https://en.wikipedia.org/wiki/New_York_City"]
)

vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

query_engine = vector_index.as_query_engine()

In [ ]:
response_vector = query_engine.query("How did New York City get its name?")

print(response_vector)

    New York City got its name in honor of the Duke of York, who later became King    James II of England. The Duke of York was appointed as the proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.


The next step involves composing questions, ideally derived from the original documents, which can be done using an LLM like GPT-4. Usually, one would then use those questions to get their answers through the RAG pipeline and evaluate them against the true answers. For the purpose of this example, we write the true answers by hand (i.e., the `eval_answers` list) and include both correct and incorrect answers so that we won’t get perfect scores in the computed metrics.

In [ ]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,000",  # incorrect answer
    "Queens",  # incorrect answer
    """New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.""",
    """New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.""",
    """The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.""",
]

eval_answers = [[a] for a in eval_answers]

This is the setup stage of the evaluation process. The competency of the `QueryEngine` is evaluated based on how well it processes and replies to these specific questions, with the responses serving as a baseline for measuring performance. The metrics from the Ragas library must be imported.

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]



The metrics list groups the metrics into a collection, which can be used in the assessment process to evaluate various elements of the QueryEngine’s performance. The results will include scores for each metric. Finally, let’s run the evaluation.

In [ ]:
from ragas.llama_index import evaluate

result = evaluate(query_engine, metrics, eval_questions, eval_answers)

# print the final scores
print(result)

    evaluating with [faithfulness]  
    100%|██████████| 1/1 [00:16<00:00, 16.95s/it]  
    evaluating with [answer_relevancy]  
    100%|██████████| 1/1 [00:03<00:00, 3.54s/it]  
    evaluating with [context_precision]  
    100%|██████████| 1/1 [00:02<00:00, 2.73s/it]  
    evaluating with [context_recall]  
    100%|██████████| 1/1 [00:07<00:00, 7.06s/it]  
    evaluating with [harmfulness]  
    100%|██████████| 1/1 [00:02<00:00, 2.16s/it]  
      
    {'faithfulness': 0.8000, 'answer_relevancy': 0.7634, 'context_precision': 0.6000,  
    'context_recall': 0.8667, 'harmfulness': 0.0000}


The system’s responses were evaluated across several metrics, each providing insight into different aspects of its performance.

First, the faithfulness score was 0.8, indicating a relatively high level of accuracy. This means the responses largely adhered to the factual content of the source material. Next, the system achieved an answer relevancy score of 0.7634. This high score suggests that most of the responses closely aligned with the intent of the given queries, ensuring that the answers were pertinent to the questions asked.

The context precision score, on the other hand, was 0.6. This indicates that while the system sometimes included relevant information, it also occasionally incorporated irrelevant context. In contrast, the context recall score was quite high, at 0.8667. This suggests that the system effectively identified and utilized the most relevant contexts in generating its final answers.

Lastly, the harmfulness score was 0, indicating that none of the responses contained harmful or inappropriate content. This metric reflects the system’s success in maintaining safe and suitable interactions.

## Custom Evaluation of RAG Pipelines

A comprehensive set of assessment benchmarks is crucial for an effective evaluation of a custom RAG system. These benchmarks are key in evaluating different aspects of the system, including its effectiveness and reliability. Employing diverse measures ensures a thorough evaluation, providing a deep understanding of the system’s overall performance. This phase involves creating a customized evaluation pipeline.

Start with loading a dataset. Construct an evaluation dataset from this initial dataset and calculate various previously mentioned metrics.

First, download the text file that can serve as the dataset. The `SimpleDirectoryReader` class is a simple utility that allows you to read local files and transform them into LlamaIndex documents.

In [ ]:
!wget 'https://raw.githubusercontent.com/idontcalculate/data-repo/main/venus_transmission.txt'

The text file can be loaded as a `Document` object, implemented by the LlamaIndex library:

In [ ]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["/content/venus_transmission.txt"])

docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

    Loaded 1 docs

In this case, the `SimpleNodeParser` tool converts documents into the structured format (the nodes). It is very useful for modifying how texts are processed. It determines the size of each text chunk, manages overlap between chunks, and adds information. Every document chunk is treated as a separate node in this system. The parser is configured explicitly with a `chunk_size` of 512, implying that each node will have 512 characters from the original document. These split portions are then used to create indexes.

In [ ]:
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(docs)
vector_index = VectorStoreIndex(nodes)

The indexes can now be used as a query engine to ask a specific question concerning the source document:

In [ ]:
query_engine = vector_index.as_query_engine()

response_vector = query_engine.query("""What was The first beings to inhabit the planet?""")
print( response_vector.response )

    The first beings to inhabit the planet were a dinoid and reptoid race from two different systems outside our solar system.

The query engine’s response is stored in the `response_vector` variable. Consequently, the document is divided into nodes, indexed, and queried using a language model. To dig deeper into the response, we can use the source_nodes key to extract the used document from the index.

In [ ]:
# First retrieved node
response_vector.source_nodes[0].get_text()

    They had heard of this beautiful new planet. At this time, Earth had two moons to harmonize the weather conditions and control the tides of the large bodies of water.  
    The first beings to inhabit the planet were a dinoid and reptoid race from two different systems outside our solar system. They were intelligent and walked on two legs like humans and were war-like considering themselves to be superior to all other life forms. In the past, the four races of humans had conflicts with them before they outgrew such behavior. They arrived on Earth to rob it of its minerals and valuable gems. Soon they had created a terrible war.  
    […]

We can also access the content of the second node that played a role in content creation by indexing the second item on the list.

In [ ]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

    Due to the radiation, the survivors of the dinoids and reptoids mutated into the Dinosaurs and giant reptilians you know of in your history. The humans that were trapped there mutated into what you call Neanderthals.  
    The Earth remained a devastated ruin, covered by a huge dark nuclear cloud and what vegetation was left was being devoured by the giant beings, also humans and animals by some. It was this way for hundreds of years before a giant comet crashed into one of the oceans and created another huge cloud. This created such darkness that the radiating heat of the Sun could not interact with Earth's gravitational field and an ice age was created. This destroyed the mutated life forms and gave the four races the chance to cleanse and heal the Earth with technology and their energy.  
    […]

You can examine the content from the second node identified as relevant by the query engine, which provides additional context or information in response to a query. This process helps understand the range of information the query engine accesses and how different segments of the indexed documents contribute to the overall response.

Creating a custom RAG evaluation process requires generating a series of questions and their corresponding answers, all related to the content we have loaded. The `generate_question_context_pairs` class uses the LLM to generate questions based on the content of each node. For every node, two questions will be generated, creating a dataset where each entry includes a context (the text of the node) and corresponding questions. This Q&A dataset will be used to assess the capabilities of a RAG system in terms of question generation and context comprehension. You can see some of the generated questions in the output.

In [ ]:
from llama_index.llms import OpenAI
from llama_index.evaluation import generate_question_context_pairs

# Define an LLM
llm = OpenAI(model="gpt-3.5-turbo")

qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=2
)

queries = list(qa_dataset.queries.values())
print(queries[0:10])

    100%|██████████| 13/13 [00:31<00:00, 2.46s/it]  
      
    ['Explain the role of different alien races in the history of our solar system according to the information provided. How did these races contribute to the transformation process and why was Earth considered a special planet?', 'Describe the advanced abilities and technology possessed by the Masters and beings mentioned in the context. How did their understanding of creation and their eternal nature shape their perspective on life and death?', 'How did the four races of humans demonstrate their mastery of creativity and what were the potential consequences of using this power for selfish reasons?', […]

The generated question-answer dataset can now be used by the `RetrieverEvaluator` class to evaluate the retriever’s performance. It uses the retriever to query each question and determines which chunks are returned as the answer. The higher the MRR and Hit rate, the better the retriever can match the chunk with the correct answer.

In [ ]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator

retriever = vector_index.as_retriever(similarity_top_k=2)

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

# Evaluate
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

display_results("OpenAI Embedding Retriever", eval_results)

| | Retriever Name | Hit Rate | MRR |  
| ---- | -------------------------- | -------- | --------- |  
| 0 | OpenAI Embedding Retriever | 0.884615 | 0.730769 |

We will improve the assessment by using new measures such as faithfulness and relevancy. To accomplish this, let’s use a portion of the created Q&A dataset and define GPT-3.5 and GPT-4 instances. It is best to use a more advanced model for evaluation while employing the less expensive model for generation to limit costs.

In [ ]:
# gpt-3.5-turbo
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_gpt35 = ServiceContext.from_defaults(llm=gpt35)

# gpt-4-turbo
gpt4 = OpenAI(temperature=0, model="gpt-4-turbo")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

vector_index = VectorStoreIndex(nodes, service_context = service_context_gpt35)
query_engine = vector_index.as_query_engine()

eval_query = queries[10]
response_vector = query_engine.query(eval_query)

print("> eval_query: ", eval_query)
print("> response_vector:", response_vector)

> eval_query: How did the colonies respond to the declaration of war by the dark forces, and what measures did they take to protect their knowledge and technology?  
> response_vector: The colonies did not fight back against the dark forces when they declared war. Instead, they sent most of their people into hiding in order to rebuild the colonies later. They also destroyed everything to ensure that their knowledge and technology would not fall into the hands of the dark forces. Additionally, Lemuria and Atlantis were destroyed by their inhabitants to prevent the misuse of their knowledge and technology by the dark forces.

We will define the evaluator classes in charge of measuring each metric. Next, we’ll use a sample response to see if it meets the test conditions.

In [ ]:
from llama_index.evaluation import RelevancyEvaluator
from llama_index.evaluation import FaithfulnessEvaluator

relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)
faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)

# Compute faithfulness evaluation

eval_result = faithfulness_gpt4.evaluate_response(response=response_vector)
# check passing parameter in eval_result if it passed the evaluation.
print( eval_result.passing )

# Relevancy evaluation
eval_result = relevancy_gpt4.evaluate_response(
    query=eval_query, response=response_vector
)
# You can check passing parameter in eval_result if it passed the evaluation.
print(eval_result.passing)

    True  
    True

We used a for-loop to feed each sample from the assessment dataset and receive the responses. In this case, we can use the LlamaIndex `BatchEvalRunner` class, which concurrently conducts the evaluation procedure in batches to be faster.

In [ ]:
#Batch Evaluator:
#BatchEvalRunner to compute multiple evaluations in batch wise manner.

from llama_index.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_gpt4, "relevancy": relevancy_gpt4},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)
# Let's get faithfulness score
faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
# get relevancy score
relevancy_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['relevancy'])

print( "> faithfulness_score", faithfulness_score )
print( "> relevancy_score", relevancy_score )

> faithfulness_score 1.0  
> relevancy_score 1.0

The batch processing method helps quickly measure the system’s performance across various queries. A faithfulness score of 1.0 indicates that the generated answers are based on the retrieved context and contain no hallucinations. Furthermore, a Relevance score of 1.0 means that the generated replies consistently fit with the collected context and queries.

However, those perfect scores are to be attributed more to the simplicity of the evaluation dataset than to the quality of the RAG system. Ideally, one would want to add more complex queries into the evaluation dataset, such as queries whose answer is distributed across different documents, or more ambiguous queries, or queries whose answers can’t be found in the dataset, etc. It’s not easy to automatically create an evaluation dataset that exactly mirrors the questions asked by users and the answers that they’d want. One way of improving the evaluation dataset is by having human expert labelers create it, which is probably the best option but also the most expensive. A cheaper but more noisy option is to improve the automatic generation of the evaluation dataset.

LangChain provides a tool called LangSmith to monitor all the inputs and outputs that go through the LLM in your application. LangSmith can greatly help you get an idea of the queries that your users ask, so that you can build an appropriate evaluation dataset based on that. The next section introduces LangSmith.